In [6]:
import importlib
import run_sg_30m
import bathtub
importlib.reload(run_sg_30m)
importlib.reload(bathtub)
import run_sg_30m as bathtubSG
from bathtub import Raster

import os
import fnmatch
import numpy as np

# Generate inundation map

In [ ]:
DEM_dir = r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\FABDEM_SG_30m"
DEM_fps = [os.path.join(DEM_dir,fp) for fp in os.listdir(DEM_dir) if fnmatch.fnmatch(fp,"SGDEM*.tif")]
DEM_fps

In [ ]:
store_dir = r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_SLR\Inundation_maps_5m"
if not os.path.exists(store_dir):
    os.mkdir(store_dir)
for DEM_fp in DEM_fps:
    try:
        bathtubSG.main(DEM_fp,slr=5.0,store_dir=store_dir)
    except Exception as e:
        print(f'{DEM_fp}: {e}')
        pass

# Import DEM

In [10]:
DEM_dir = r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\FABDEM_SG_30m"
DEM_fps = [os.path.join(DEM_dir,fp) for fp in os.listdir(DEM_dir) if fnmatch.fnmatch(fp,"SGDEM*.tif")]
DEM_fps

['C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\FABDEM_SG_30m\\SGDEM0.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\FABDEM_SG_30m\\SGDEM1.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\FABDEM_SG_30m\\SGDEM10.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\FABDEM_SG_30m\\SGDEM11.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\FABDEM_SG_30m\\SGDEM12.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\FABDEM_SG_30m\\SGDEM13.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\FABDEM_SG_30m\\SGDEM14.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\FABDEM_SG_30m\\SGDEM15.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\FABDEM_SG_30m\\SGDEM16.tif',
 'C:\\Users\

## Calculate total land area

In [14]:
total_area = 0
for fp in DEM_fps:
    try:
        raster = Raster(fp)
        dem = raster.data(nodata=True)
        # print(raster.shape,dem[dem>0].shape,dem[~np.isnan(dem)].shape)
        total_area += dem[~np.isnan(dem)].shape[0]
    except Exception as e:
        print(f'{fp}: {e}')

total_area = (total_area * 30*30)/(10**6)
print(f'Total land area of singapore: {total_area} km2')

Total land area of singapore: 744.9543 km2


# Calculate inundation area from SLR=4.0m

In [3]:
slr_4m_dir = r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_SLR\Inundation_maps"
slr_4m_fps = [os.path.join(slr_4m_dir,fp) for fp in os.listdir(slr_4m_dir) if fnmatch.fnmatch(fp,"SGDEM*.tif")]
slr_4m_fps

['C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps\\SGDEM10_slr_4.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps\\SGDEM11_slr_4.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps\\SGDEM12_slr_4.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps\\SGDEM13_slr_4.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps\\SGDEM14_slr_4.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps\\SGDEM15_slr_4.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps\\SGDEM16_slr_4.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps\\SGDEM17_slr_

In [17]:
inundated_4m = 0
for fp in slr_4m_fps:
    try:
        raster = Raster(fp)
        dem = raster.data(nodata=True)
        # print(raster.shape,dem[dem>0].shape,dem[~np.isnan(dem)].shape)
        inundated_4m += dem[~np.isnan(dem)].shape[0]

    except Exception as e:
        print(f'{fp}: {e}')

inundated_4m = (inundated_4m * 30*30)/(10**6)
print(f'Total inundated land area of singapore: {inundated_4m} km2')
print(f'Percentage land area inundated: {inundated_4m/total_area*100:.2f}%')

Total inundated land area of singapore: 47.9349 km2
Percentage land area inundated: 6.43%


## Convert to binary files

In [44]:
save_dir = r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_SLR\Inundation_maps"
save_dir = os.path.join(save_dir,"flooded_bool")
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
for fp in slr_4m_fps:
    try:
        raster = Raster(fp)
        # dem = raster.data(nodata=True)
        # print(raster.shape,dem[dem>0].shape,dem[~np.isnan(dem)].shape)
        # inundated_4m += dem[~np.isnan(dem)].shape[0]
        flooded_raster = raster.__gt__(0)
        # flooded_raster = flooded_raster.astype(np.uint8)
        filename = os.path.basename(fp)
        filename = os.path.join(save_dir,filename)
        # astype method returns a raster
        flooded_binary = flooded_raster.astype(int)
        # writeNodata method does not return
        flooded_binary.writeNodata(0)
        flooded_binary.writeRaster(filename, dtype = 'int8') # cast boolean to binary, and set nodata as 0

    except Exception as e:
        print(f'{fp}: {e}')

    

C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_SLR\Inundation_maps\SGDEM36_slr_4.tif: Deleting C:/Users/hypak/OneDrive - Singapore Management University/Documents/Data/SG_SLR/Inundation_maps/flooded_bool/SGDEM36_slr_4.tif failed: Permission denied


# Calculate inundation area from SLR=5.0m

In [4]:
slr_5m_dir = r"C:\Users\hypak\OneDrive - Singapore Management University\Documents\Data\SG_SLR\Inundation_maps_5m"
slr_5m_fps = [os.path.join(slr_5m_dir,fp) for fp in os.listdir(slr_5m_dir) if fnmatch.fnmatch(fp,"SGDEM*.tif")]
slr_5m_fps

['C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps_5m\\SGDEM10_slr_5.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps_5m\\SGDEM11_slr_5.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps_5m\\SGDEM12_slr_5.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps_5m\\SGDEM13_slr_5.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps_5m\\SGDEM14_slr_5.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps_5m\\SGDEM15_slr_5.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundation_maps_5m\\SGDEM16_slr_5.tif',
 'C:\\Users\\hypak\\OneDrive - Singapore Management University\\Documents\\Data\\SG_SLR\\Inundati

In [18]:
inundated_5m = 0
for fp in slr_5m_fps:
    try:
        raster = Raster(fp)
        dem = raster.data(nodata=True)
        # print(raster.shape,dem[dem>0].shape,dem[~np.isnan(dem)].shape)
        inundated_5m += dem[~np.isnan(dem)].shape[0]

    except Exception as e:
        print(f'{fp}: {e}')

inundated_5m = (inundated_5m * 30*30)/(10**6)
print(f'Total inundated land area of singapore: {inundated_5m} km2')
print(f'Percentage land area inundated: {inundated_5m/total_area*100:.2f}%')

Total inundated land area of singapore: 110.0349 km2
Percentage land area inundated: 14.77%
